**Multiclass Fish Image Classification using Pre-trained models**

**VGG16**

In [ ]:
#import the required libraries

import os #manage path, folder, files

import torch #building and training deeplearning model
import torch.nn as nn # layer, activation & loss function
from PIL import Image
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#set the device
#this means you'll run your model on GPU

if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device('cpu')


In [ ]:
#transformation for training data
train_transform = transforms.Compose(
    [
        transforms.RandomResizedCrop(224, scale = (0.8, 1.0)), #random zoom + crop
        transforms.RandomHorizontalFlip(), #  flip the image (right ---> left)
        transforms.RandomRotation(15), # random rotation clockwise or counter clockwise
        transforms.ToTensor()   # #rescale to [0,1]
    ])

In [ ]:
 #for test and val
# data preprocessing

test_val_transform = transforms.Compose(
    [
        transforms.Resize((224, 224)), #resize piexel same as training
        transforms.ToTensor() #convert image to tensor
    ]
)

In [ ]:
# load the datasets
train_datasets = datasets.ImageFolder('/content/drive/MyDrive/images.cv_jzk6llhf18tm3k0kyttxz/data/train', transform = train_transform)
test_datasets = datasets.ImageFolder('/content/drive/MyDrive/images.cv_jzk6llhf18tm3k0kyttxz/data/test', transform = test_val_transform)
val_datasets = datasets.ImageFolder('/content/drive/MyDrive/images.cv_jzk6llhf18tm3k0kyttxz/data/val', transform = test_val_transform)

In [ ]:
train_datasets

Dataset ImageFolder
    Number of datapoints: 6225
    Root location: /content/drive/MyDrive/images.cv_jzk6llhf18tm3k0kyttxz/data/train
    StandardTransform
Transform: Compose(
               RandomResizedCrop(size=(224, 224), scale=(0.8, 1.0), ratio=(0.75, 1.3333), interpolation=bilinear, antialias=True)
               RandomHorizontalFlip(p=0.5)
               RandomRotation(degrees=[-15.0, 15.0], interpolation=nearest, expand=False, fill=0)
               ToTensor()
           )

In [ ]:
# create the dataloader
train_data_loader = DataLoader(train_datasets, batch_size = 5, shuffle = True)
test_data_loader = DataLoader(test_datasets, batch_size = 5, shuffle = True)
val_data_loader = DataLoader(val_datasets, batch_size = 5, shuffle = True)


In [ ]:
# load the pre-trained  model VGG16
model = models.vgg16(pretrained = True)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:03<00:00, 153MB/s]


In [ ]:
model.classifier[6] =nn.Linear(4096, 11)
model = model.to(device)

In [ ]:
#loss function and optimizer

cl = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier.parameters(),lr = 0.001)

In [ ]:
# train the model

epochs = 25

for i in range(epochs):
  model.train()
  total_loss = 0.0 # total running loss
  total = 0 #  #  Total number of images evaluated
  correct = 0 #  calculate how many predictions correct

  for img, tar in train_data_loader:
    img, tar = img.to(device), tar.to(device)
    optimizer.zero_grad()
    outputs = model(img)
    loss = cl(outputs, tar)
    loss.backward()
    optimizer.step()
    total_loss += loss.item()
    _, preds = torch.max(outputs, 1)
    correct += (preds == tar).sum().item()
    total += tar.size(0)

  print(f"Epoch [{i + 1}/{epochs}], loss {total_loss:.4f}")


Epoch [1/25], loss 3553.0251
Epoch [2/25], loss 2978.3064
Epoch [3/25], loss 2059.6943
Epoch [4/25], loss 2369.0254
Epoch [5/25], loss 2153.2963
Epoch [6/25], loss 2199.5933
Epoch [7/25], loss 2013.1498
Epoch [8/25], loss 1840.0094
Epoch [9/25], loss 1857.6461
Epoch [10/25], loss 1791.8885
Epoch [11/25], loss 1432.2816
Epoch [12/25], loss 1876.3113
Epoch [13/25], loss 1769.0886
Epoch [14/25], loss 1659.1003
Epoch [15/25], loss 1516.8948
Epoch [16/25], loss 1196.7696
Epoch [17/25], loss 1191.1553
Epoch [18/25], loss 1801.6364
Epoch [19/25], loss 1612.4228
Epoch [20/25], loss 1331.4715
Epoch [21/25], loss 1519.3809
Epoch [22/25], loss 1091.9782
Epoch [23/25], loss 1501.5024
Epoch [24/25], loss 1192.9570
Epoch [25/25], loss 934.9310


Evlauate validation data

In [ ]:
model.eval()

all_preds = []
all_tars = []

with torch.no_grad():
  for img, tar in val_data_loader:
    img, tar = img.to(device), tar.to(device)
    outputs = model(img)
    _, preds = torch.max(outputs, 1)
    all_preds.extend(preds.cpu().numpy())
    all_tars.extend(tar.cpu().numpy())


  #accuracy
  accuracy = accuracy_score(all_tars, all_preds)
  print(f"val_accuracy: {accuracy*100 :.2f}%")

  #precision
  precision = precision_score(all_tars , all_preds, average = 'macro')
  print(f"val_precision: {precision*100 :.2f}%")

  # recall
  recall = recall_score(all_tars , all_preds, average = 'macro')
  print(f"val_recall: {recall*100 :.2f}%")


#f1 score
  f1score = f1_score(all_tars, all_preds, average = 'macro')
  print(f"val f1_score: {f1score *100:.2f}%")


# Calculate confusion matrix
  cm = confusion_matrix(all_tars, all_preds)
  print('confusion matrix')
  print(cm)

val_accuracy: 98.44%
val_precision: 89.66%
val_recall: 90.33%
val f1_score: 89.99%
confusion matrix
[[186   0   0   0   0   0   0   0   0   1   0]
 [  7   0   0   0   0   0   0   0   0   3   0]
 [  0   0 105   0   0   0   0   0   0   0   0]
 [  0   0   0  94   0   0   0   0   0   0   0]
 [  0   0   0   0  97   0   0   0   0   0   0]
 [  0   0   0   0   0  90   0   0   0   0   0]
 [  0   0   0   2   0   0 111   0   0   0   0]
 [  0   0   0   1   0   0   0  95   0   1   0]
 [  0   0   0   0   0   0   0   0 100   0   0]
 [  0   0   0   0   0   2   0   0   0  99   0]
 [  0   0   0   0   0   0   0   0   0   0  98]]


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# Evaluation on test data

model.eval()

all_preds = []
all_tars = []

with torch.no_grad():
  for img, tar in test_data_loader:
    img, tar = img.to(device), tar.to(device)
    outputs = model(img)
    _, preds = torch.max(outputs, 1)

    all_preds.extend(preds.cpu().numpy())
    all_tars.extend(tar.cpu().numpy())


# accuracy
  accuracy = accuracy_score(all_tars, all_preds)
  print(f"Test Accuracy: {accuracy *100:.2f}%")

# precision
  precision = precision_score(all_tars, all_preds, average = 'macro')
  print(f"Test precision: {precision *100:.2f}%")

#recall
  recall = recall_score(all_tars, all_preds, average = 'macro')
  print(f"Test recall: {recall *100:.2f}%")


#f1 score
  f1score = f1_score(all_tars, all_preds, average = 'macro')
  print(f"Test f1_score: {f1score *100:.2f}%")


# Calculate confusion matrix
  cm = confusion_matrix(all_tars, all_preds)
  print('confusion matrix')
  print(cm)





Test Accuracy: 98.59%
Test precision: 89.81%
Test recall: 89.95%
Test f1_score: 89.86%
confusion matrix
[[520   0   0   0   0   0   0   0   0   0   0]
 [ 13   0   0   0   0   0   0   0   0   0   0]
 [  0   0 296   0   0   0   0   0   0   2   0]
 [  0   0   0 305   0   0   0   0   0   0   0]
 [  0   0   0   0 286   0   0   0   0   0   0]
 [  0   0   0   0   0 289   0   0   0   2   0]
 [  1   0   0   1   3   0 265   0   0   3   0]
 [  4   0   0   2   0   0   0 312   0   9   0]
 [  0   0   0   0   0   0   0   0 288   1   0]
 [  1   0   1   0   1   1   0   0   0 289   0]
 [  0   0   0   0   0   0   0   0   0   0 292]]


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# evaluate the train data

model.eval()

all_preds = []
all_tars = []


with torch.no_grad():
  for img, tar in train_data_loader:
    img, tar = img.to(device), tar.to(device)
    outputs = model(img)
    _,preds = torch.max(outputs, 1)

    all_preds.extend(preds.cpu().numpy())
    all_tars.extend(tar.cpu().numpy())


  # accuracy
  accuracy = accuracy_score(all_tars, all_preds)
  print(f"Train Accuracy: {accuracy* 100:.2f}%")

  #precision
  precision = precision_score(all_tars, all_preds, average = 'macro')
  print(f"Train precision: {precision*100:.2f}%")

  #recall
  recall = recall_score(all_tars, all_preds, average = 'macro')
  print(f"Train recall:{recall*100:.2f}%")


  #f1 score
  f1score = f1_score(all_tars, all_preds, average = 'macro')
  print(f" f1_score: {f1score *100:.2f}%")


# Calculate confusion matrix
  cm = confusion_matrix(all_tars, all_preds)
  print('confusion matrix')
  print(cm)

Train Accuracy: 98.35%
Train precision: 89.45%
Train recall:89.74%
 f1_score: 89.58%
confusion matrix
[[1094    0    0    0    0    0    0    0    1    1    0]
 [  16    0    0    0    0    0    0    1    0   13    0]
 [   0    0  555    0    2    0    0    3    0    9    0]
 [   0    0    0  566    0    0    0    0    0    0    0]
 [   0    0    0    2  567    0    0    0    0    4    0]
 [   0    0    0    0    0  579    0    0    0    0    0]
 [   3    0    0    8    0    0  553    0    0    7    0]
 [   0    0    1    2    0    0    0  522    0   13    0]
 [   1    0    0    0    0    0    0    0  572    3    0]
 [   1    0    2    2    2    1    0    1    3  535    0]
 [   0    0    0    0    0    0    0    1    0    0  579]]


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# save the model

torch.save(model.state_dict(), 'vgg16_fish_model.pth')